# **Large Language Models (LLMS)**
---

### *Import Libraries*

In [1]:
import os
import openai

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

## Setup OpenAI API Key 
---

In [3]:
#LLM: takes string as input and returns and string 
#ChatModels: takes a list of messages as input and returns a message 

llm = OpenAI(openai_api_key=key_xyz)
chat_model = ChatOpenAI(openai_api_key=key_xyz)

In [4]:
#openai.api_key = os.environ["OPENAI_API_KEY"]

## Language Models Within LangChain: *LLMs and ChatModels*
---

#### Two Types of Language Models Within LangChain: 
- **LLMs**: input = string -> output = string 
    - to exit out `"\n\n"` from string when printing, use `[2:]`
- **ChatModels**: input = list of `ChatMessage`s -> output = `ChatMessage`
    - `ChatMessage` has two components: 
        - `content` *(content of message)* 
        - `role` *(role of entity the `ChatMessage` is coming from)*
            > `role` Objects: 
            > - `HumanMessage` = from a Human/User 
            > - `AIMessage` = from AI/Assistant
            > - `SystemMessage` = from system 
            > - `FunctionMessage` = from a function cell
            > - custom role class

In [5]:
example1 = "Hello!"
example_sentence = "What would be a good name for a company that made colorful socks?"

#### `predict`: takes a string -> returns a string 

In [6]:
# predict: takes a string -> returns a string 

print("example string: {} \n ------ \n LLM: {} \n ------ \n ChatModel: {}".format(example1,llm.predict(example1)[2:],chat_model.predict(example1)))

example string: Hello! 
 ------ 
 LLM: Welcome to the forum. How are you doing today? 
 ------ 
 ChatModel: Hello! How can I assist you today?


#### `predict_messages`: takes a list of messages -> returns a message
- `ChatMessage`: 
    - `content` = string 
    - `role` = `HumanMessage` *from a human/user*

In [7]:
# predict_messages: takes a list of messages -> returns a message
# HumanMessage role 
from langchain.schema import HumanMessage

messages = [HumanMessage(content=example_sentence)]


llm_string1 = llm.predict_messages(messages)
cm_string1 = chat_model.predict_messages(messages)

print("String Input: {} \n ------ \n LLM: {} \n   {} \n ------ \n ChatModel: {} \n   {}".format(messages, llm_string1, llm_string1.content[2:], cm_string1, cm_string1.content))

String Input: [HumanMessage(content='What would be a good name for a company that made colorful socks?', additional_kwargs={}, example=False)] 
 ------ 
 LLM: content='\n\nSockorific!' additional_kwargs={} example=False 
   Sockorific! 
 ------ 
 ChatModel: content='Socktastic' additional_kwargs={} example=False 
   Socktastic


## Prompt Templates: 
---
- most LLM appications do not pass user input directly into LLM
    - will instead add user input to a larger piece of text, *prompt template* to provide additional context 
- `PromptTemplate` bundles logic from user input into a fully formatted prompt 

> Advantages Of `PromptTemplate` Over Raw String Formats: 
> - ability to "partial" out variables (focus on a single variable or a few)
> - easy to combine templates into a single prompt  
> - used to produce a list of messages (contains information about content but also each message) 
>   - common for a ChatPromptTemplate to be a list of ChatMessageTemplates each containing instructions for how to format the ChatMessage

In [8]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [10]:
from langchain.prompts.chat import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])
chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming!")

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming!', additional_kwargs={}, example=False)]

## Output Parsers: 
---
- convert raw output of a LLM into a format that can be used downstream 

> Main Types of `OutputParsers`: 
> - convert text from LLM -> structured information (ex: JSON)
> - convert a `ChatMessage` into a string 
> - convert extra information returned from a call besides the message into a string 

In [11]:
# convert comma separated list into a list 
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser): 
    """Parse Output of LLM Call Into A Comma-Separated List"""
    def parse(self,text:str): 
        '''Parse Output of LLM Call'''
        return text.strip().split(", ")
    
CommaSeparatedListOutputParser().parse("hi, how are ya, bye")

['hi', 'how are ya', 'bye']

## LLMChain 
--- 
- combine everything into one chain, bundling a modular piece of logic 
- chain will accept input variables 
    - variables passed to a prompt template to create a prompt 
        - prompt passed to a LLM 
            - LLM output passed thrugh an *(optional)* output parser

In [12]:
# LangChain Imports 

#from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser


In [13]:
class CommaSeparatedListOutputParser(BaseOutputParser): 

    def parse(self, text:str): 
        return text.strip().split(", ")
    
template = """You are a helpful assistant who generates comma separated lists. 
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list. 
ONLY return a comma separated list, and nothing more."""

system_message_prompt = SystemMessagePromptTemplate.from_template(template) 
human_template = "{text}" 
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template) 

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


#FIX API KEY LATER 
chain = LLMChain(llm=ChatOpenAI(openai_api_key=key_xyz),
                 prompt = chat_prompt, 
                 output_parser=CommaSeparatedListOutputParser()
                 )


chain.run("colors")

['red', 'blue', 'green', 'yellow', 'orange']